In [ ]:
!pip install TTS

In [ ]:
dataset_path = '../input/the-lj-speech-dataset/LJSpeech-1.1/'

In [ ]:
!pwd

In [ ]:
%cd ..
%pwd

In [ ]:
!ls

In [ ]:
%cd working

In [ ]:
%pwd

In [ ]:
output_path = './output/fakeljspeech/'

In [ ]:
import os

# Trainer: Where the ✨️ happens.
# TrainingArgs: Defines the set of arguments of the Trainer.
from trainer import Trainer, TrainerArgs

# GlowTTSConfig: all model related values for training, validating and testing.
from TTS.tts.configs.glow_tts_config import GlowTTSConfig

# BaseDatasetConfig: defines name, formatter and path of the dataset.
from TTS.tts.configs.shared_configs import BaseDatasetConfig
from TTS.tts.datasets import load_tts_samples
from TTS.tts.models.glow_tts import GlowTTS
from TTS.tts.utils.text.tokenizer import TTSTokenizer
from TTS.utils.audio import AudioProcessor

In [ ]:
dataset_config = BaseDatasetConfig(
    formatter="ljspeech", meta_file_train="metadata.csv", path=dataset_path
)

In [ ]:
config = GlowTTSConfig(
    batch_size=32,
    eval_batch_size=16,
    num_loader_workers=4,
    num_eval_loader_workers=4,
    run_eval=True,
    test_delay_epochs=-1,
    epochs=1000,
    text_cleaner="phoneme_cleaners",
    use_phonemes=True,
    phoneme_language="en-us",
    phoneme_cache_path=os.path.join(output_path, "phoneme_cache"),
    print_step=25,
    print_eval=False,
    mixed_precision=True,
    output_path=output_path,
    datasets=[dataset_config],
)

In [ ]:
# INITIALIZE THE AUDIO PROCESSOR
# Audio processor is used for feature extraction and audio I/O.
# It mainly serves to the dataloader and the training loggers.
ap = AudioProcessor.init_from_config(config)

# INITIALIZE THE TOKENIZER
# Tokenizer is used to convert text to sequences of token IDs.
# If characters are not defined in the config, default characters are passed to the config
tokenizer, config = TTSTokenizer.init_from_config(config)

# LOAD DATA SAMPLES
# Each sample is a list of ```[text, audio_file_path, speaker_name]```
# You can define your custom sample loader returning the list of samples.
# Or define your custom formatter and pass it to the `load_tts_samples`.
# Check `TTS.tts.datasets.load_tts_samples` for more details.
train_samples, eval_samples = load_tts_samples(
    dataset_config,
    eval_split=True,
    eval_split_max_size=config.eval_split_max_size,
    eval_split_size=config.eval_split_size,
)

# INITIALIZE THE MODEL
# Models take a config object and a speaker manager as input
# Config defines the details of the model like the number of layers, the size of the embedding, etc.
# Speaker manager is used by multi-speaker models.
model = GlowTTS(config, ap, tokenizer, speaker_manager=None)

# INITIALIZE THE TRAINER
# Trainer provides a generic API to train all the 🐸TTS models with all its perks like mixed-precision training,
# distributed training, etc.
trainer = Trainer(
    TrainerArgs(), config, output_path, model=model, train_samples=train_samples, eval_samples=eval_samples
)

# AND... 3,2,1... 🚀
trainer.fit()

In [ ]:
!tts -h

In [ ]:
!tts

In [ ]:
from TTS.api import TTS

# List available 🐸TTS models and choose the first one
model_name = TTS.list_models()[0]

In [ ]:
TTS.list_models()

In [ ]:
# Init TTS
tts = TTS(model_name)
# Run TTS
# ❗ Since this model is multi-speaker and multi-lingual, we must set the target speaker and the language
# Text to speech with a numpy output
wav = tts.tts("This is a test! This is also a test!!", speaker=tts.speakers[0], language=tts.languages[0])
# Text to speech to a file
tts.tts_to_file(text="Hello world!", speaker=tts.speakers[0], language=tts.languages[0], file_path="output.wav")

In [ ]:
output_folder = './fake_speeches/'

In [ ]:
# Init TTS with the target model name
tts = TTS(model_name="tts_models/en/ljspeech/tacotron2-DDC", progress_bar=False, gpu=True)
# Run TTS
tts.tts_to_file(text="Hello World", file_path='output_1.wav')

In [ ]:
dataset_path = '../input/the-lj-speech-dataset/LJSpeech-1.1/'

In [ ]:
!mkdir fake_speeches

In [ ]:
import csv

with open(dataset_path + 'metadata.csv') as csv_file:
    csv_reader = csv.reader(csv_file, delimiter='|')
    line_count = 0
    for row in csv_reader:
        try:
            text_from_metadata=row[2]
            tts.tts_to_file(text=text_from_metadata, file_path='fake_speeches/fake_{}.wav'.format(line_count))
        except:
            print('problem processing line: {}'.format(line_count))
        line_count+=1
        if line_count == 2000:
            break
    print(f'Processed {line_count} lines.')

In [3]:
import shutil


In [6]:
output_filename = 'fake_speeches_zipped.zip'

In [7]:
!pwd

/kaggle/working


In [8]:
!ls

__notebook_source__.ipynb  output      output_1.wav
fake_speeches		   output.wav  state.db


In [9]:
dir_name = './fake_speeches/'

In [10]:
shutil.make_archive(output_filename, 'zip', dir_name)

'/kaggle/working/fake_speeches_zipped.zip.zip'